In [ ]:
import numpy as np
import random
from numpy import *
import pickle

max_iter = 200000
#print (len(Board))
Qtable = {}
exp_fac = 0.9
gamma = 0.9

def winner(player):
    win = [[[0,0],[0,1],[0,2]],[[1,0],[1,1],[1,2]],[[2,0],[2,1],[2,2]],[[0,0],[1,0],[2,0]],[[0,1],[1,1],[2,1]],[[0,2],[1,2],[2,2]],[[0,0],[1,1],[2,2]],[[0,2],[1,1],[2,0]]]
    count = 0
    for j in range(len(win)):
        count = 0
        for k in range(len(player)):
            for l in range(len(win[j])):
                if player[k] == win[j][l]:
                    count = count + 1
                    if count == 3:
                        return 1;
                        break
            else:
                continue
            break
        else:
            continue
        break
    return 0;

def getstate(availablemoves,player):
    length = len(availablemoves)
    s = ''
    for i in range(3):
        for j in range(3):
            if [i,j] in availablemoves:
                s = s + '9'
            elif [i,j] in player:
                s = s + 'X'
            else:
                s = s + 'O'
    return s


def get_move_and_reward(player_move,reward):
    s = ''
    for i in range(3):
        for j in range(3):
            if [i,j] == player_move:
                s = s + 'X'
            else:
                s = s + 'O'
    s = s + ',' + str(double(reward))
    return s

def get_move_from_key(key):
    move_key = key.split(',')[0]
    idx = move_key.index('X')
    (x,y) = divmod(idx,3)
    move = [x,y]
    return move
    

def make_key(state,value):     
    if state in Qtable.keys():
        move = value.split(',')[0]
        list_values = Qtable[state]
        update_flag = 0
        for i in range(len(list_values)):
            temp = list_values[i].split(',')[0]
            if move == temp:
                update_flag = 1
                ext_value = list_values[i].split(',')[1]
                break
                
        if update_flag == 1:
            updated_value = Qlearn(state,value,ext_value,Qtable)
            del Qtable[state][i]
            Qtable[state].append(value.split(',')[0] +','+str(updated_value))
        else:
            Qtable[state].append(value)
    else:
        Qtable[state]=[]
        Qtable[state].append(value)

def Qlearn(state, value,ext_value, Qtable): 
    idx = value.index('X') 
    state = list(state)
    state[idx]='X'
    next_state ="".join(state)
    if next_state in Qtable.keys():
        values = Qtable[next_state]
        max_val = double(values[0].split(',')[1])
        if len(values)>1:
            for i in range(1,len(values)):
                curr_val = double(values[i].split(',')[1])
                if curr_val > max_val:
                    max_val = curr_val               
    else:
        max_val = 0
    updated_value = double(value.split(',')[1]) + double(ext_value) + double(gamma * max_val)
    return updated_value 

def best_move(state):
    values = Qtable[state]
    player_move = values[0]
    max_val = double(values[0].split(',')[1])
    if len(values)>1:
        for j in range(1,len(values)):
            curr_val = double(values[j].split(',')[1])
            if curr_val > max_val:
                max_val = curr_val 
                player_move = values[j]
    return player_move

def best_moveO(state):
    values = Qtable[state]
    player_move = values[0]
    min_val = double(values[0].split(',')[1])
    if len(values)>1:
        for j in range(1,len(values)):
            curr_val = double(values[j].split(',')[1])
            if curr_val < min_val:
                min_val = curr_val 
                player_move = values[j]
    return player_move

for iteration in range(max_iter):
    Board = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]
    player1 = [[] for z in range(5)]
    player2 = [[] for z in range(5)]
    if random.random() < 0.5:
        start_player = 'player1' 
    else:
        start_player = 'player2'

    unoccupied = Board
    for i in range(9):
        if start_player == 'player1':
            if i==0 or int(i%2)==0:
                num = np.random.random()
                state = getstate(unoccupied,player1)
                if (num < exp_fac) and (state in Qtable.keys()): 
                    player1_move_key = best_move(state)
                    player1_move = get_move_from_key(player1_move_key)
                    if i==0:
                        player1[i] = player1_move
                    else:
                        player1[int(i/2)] = player1_move    
                else:
                    idx = np.random.randint(len(unoccupied), size=1)
                    idx = idx[0]
                    player1_move = unoccupied[idx]
                    if i==0:
                        player1[i] = player1_move
                    else:
                        player1[int(i/2)] = player1_move
                
                win1 = winner(player1)
                unoccupied.remove(player1_move)                                  #loop1
                if win1 == 1:
                    reward = 1
                    value = get_move_and_reward(player1_move,reward)
                    make_key(state,value)
                    break
        
            elif int(i%2)==1:
                idx = np.random.randint(len(unoccupied), size=1)
                idx = idx[0]
                player2_move = unoccupied[idx]
                if i==1:
                    player2[0] = player2_move
                else:
                    player2[int((i-1)/2)] = player2_move
                    
                win2 = winner(player2)
                unoccupied.remove(player2_move)
                if win2 == 1:
                    reward = -1
                    value = get_move_and_reward(player1_move,reward)
                    make_key(state,value) #previous state with this reward values
                    break
                   
            next_state = getstate(unoccupied,player1)
            if next_state in Qtable.keys():
                rand_key = best_moveO(next_state)
                exp = rand_key.split(',')[1]
                reward = reward + double(0.01) * double(exp)
            else:
                reward = 0
            value = get_move_and_reward(player1_move,reward)
            make_key(state,value)
        
        elif start_player == 'player2':
            if i==0 or int(i%2)==0:
                idx = np.random.randint(len(unoccupied), size=1)
                idx = idx[0]
                player2_move = unoccupied[idx]
                if i==0:
                    player2[i] = player2_move
                else:
                    player2[int(i/2)] = player2_move
                    
                win2 = winner(player2)
                unoccupied.remove(player2_move)

                if win2 == 1:
                    reward = -1
                    value = get_move_and_reward(player1_move,reward)
                    make_key(state,value)
                    break
   
            elif int(i%2)==1:
                num = np.random.random()
                state = getstate(unoccupied,player1)
                if (num < exp_fac) and (state in Qtable.keys()):
                    player1_move_key = best_move(state)                                
                    player1_move = get_move_from_key(player1_move_key)
                    if i==0:
                        player1[i] = player1_move
                    else:
                        player1[int(i/2)] = player1_move   
                        
                else:
                    idx = np.random.randint(len(unoccupied), size=1)
                    idx = idx[0]
                    player1_move = unoccupied[idx]
                    if i==1:
                        player1[0] = player1_move
                    else:
                        player1[int((i-1)/2)] = player1_move
                
                win1 = winner(player1)
                unoccupied.remove(player1_move)                   
                if win1 == 1:
                    reward = 1
                    value = get_move_and_reward(player1_move,reward)
                    make_key(state,value)
                    break
                    
            if i > 1:
                next_state = getstate(unoccupied,player1)
                if next_state in Qtable.keys():
                    rand_key = best_moveO(next_state)
                    exp = rand_key.split(',')[1]
                    reward = reward + double(0.01) * double(exp)
                else:
                    reward = 0
                value = get_move_and_reward(player1_move,reward)
                make_key(state,value)
    if win1!=1 and win2!=1:
        reward = 0.0
        value = get_move_and_reward(player1_move,reward)
        make_key(state,value) 


pickle_out = open("Qtable.pickle","wb")
pickle.dump(Qtable, pickle_out)
pickle_out.close()